In [1]:
import numpy as np

In [141]:
class SparseMatrix:
    def __init__(self, n, m, elems=[]):
        """
        Arguments:
        * n, m --- matrix size (n rows, m columns)
        * elems: a list of triplets (r, c, v) 
            --- represents that row r and column c of the matrix contains value v.
            All (r, c) pairs must be distinct, 0-indexed.
        """
        self._elems = elems.copy()
        self._n = n
        self._m = m
        
    @staticmethod
    def from_numpy(mat: np.ndarray) -> SparseMatrix:
        """
        Constructs a SparseMatrix from a numpy matrix
        """
        if len(mat.shape) != 2: 
            raise IllegalArgumentException("mat must be a 2-d numpy array")
        return SparseMatrix(mat.shape[0], mat.shape[1], 
                            [(r, c, mat[r, c]) 
                             for r in range(mat.shape[0]) 
                             for c in range(mat.shape[1]) 
                             if mat[r, c] != 0]
                           )
    
    def transpose(self) -> SparseMatrix:
        return SparseMatrix(self._m, self._n, [
            (c, r, v) for (r, c, v) in self._elems
        ])
    
    def to_numpy(self) -> np.ndarray:
        result = np.zeros((self._n, self._m))
        for (r, c, v) in self._elems:
            result[r, c] = v
        return result
    
    def mat_vec_mul(self, vec: np.ndarray) -> np.ndarray:
        """
        Multiples the SparseMatrix by a vector (1-d numpy.ndarray), returns 1-d numpy.ndarray
        """
        if len(vec.shape) != 1: 
            raise IllegalArgumentException("vec is not a 1-d np.ndarray")
        if vec.shape[0] != self._m: 
            raise IllegalArgumentException("vec's length does not match number of columns")
        result = np.zeros(self._n)
        for (r, c, v) in self._elems:
            result[r] += v * vec[c]
        return result
    
    def vec_mat_mul(self, vec: np.ndarray) -> np.ndarray:
        """
        Multiples a vector (1-d numpy.ndarray) by the SparseMatrix, returns 1-d numpy.ndarray
        """
        if len(vec.shape) != 1: 
            raise IllegalArgumentException("vec is not a 1-d np.ndarray")
        if vec.shape[0] != self._n: 
            raise IllegalArgumentException("vec's length does not match number of rows")
        result = np.zeros(self._m)
        for (r, c, v) in self._elems:
            result[c] += v * vec[r]
        return result

In [142]:
to_sparse = SparseMatrix.from_numpy
assert(np.all(
    to_sparse(np.array([[1, 1], [0, 1]])).to_numpy()
    ==
    np.array([[1, 1], [0, 1]])
))
assert(
    len(to_sparse(np.array([[1, 1], [0, 1]]))._elems) == 3
)
assert(np.all(
    to_sparse(np.array([[1, 1], [0, 1]])).mat_vec_mul(np.array([2, 3]))
    ==
    np.array([5, 3])
))
assert(np.all(
    to_sparse(np.array([[1, 1], [0, 1]])).vec_mat_mul(np.array([2, 3]))
    ==
    np.array([2, 5])
))
assert(np.all(
    to_sparse(np.array([[1, 1], [0, 2]])).transpose().to_numpy()
    ==
    np.array([[1, 0], [1, 2]])
))

$\|Ax - b\| \to \min \Leftrightarrow \frac{1}{2} x^T(2A^T A) x - (2A^T b)^Tx + b^Tb \to \min$

In [146]:
def conjugate_gradient(A, b):
    x = x_0
    v = (A @ x - b)
    d = v
    v_norm = np.dot(v, v)
    
    result = [x.copy()]
    for i in range(len(b)):
        Ad = np.dot(A, d)
        alpha = v_norm / np.dot(d, Ad)
        x = x - alpha * d
        v = v - alpha * Ad
        v_norm_new = np.dot(v, v)

        d = v + (v_norm_new / v_norm) * d
        v_norm = v_norm_new
        result.append(x.copy())
    return result

def solve(A: SparseMatrix, b):
    # A -> A^T A
    # b -> A^T b
    At = A.transpose()
    x = np.zeros(b.shape)
    v = At.mat_vec_mul(-b)
    d = v
    v_norm = np.dot(v, v)
    
    for i in range(len(b)):
        # Ad = A_before @ d = (A^T A) d
        Ad = At.mat_vec_mul(A.mat_vec_mul(d))
        alpha = v_norm / np.dot(Ad, d)
        x = x - alpha * d
        v = v - alpha * Ad
        v_norm_new = np.dot(v, v)

        d = v + (v_norm_new / v_norm) * d
        v_norm = v_norm_new
    return x

In [147]:
def test(dim=5):
    from scipy.linalg import sqrtm
    A = np.random.rand(dim, dim)
    b = np.random.rand(dim)
    
    solve_res = solve(SparseMatrix.from_numpy(A), b)
    exact_res, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    
    norm = lambda x: (A @ x - b).T @ (A @ x - b)
    
    print('solve: {}, {}'.format(norm(solve_res), solve_res))
    print('exact: {}, {}'.format(norm(exact_res), exact_res))
    
    assert(np.isclose(norm(solve_res), norm(exact_res)))

In [156]:
test(dim=5)

solve: 2.2431972170590166e-20, [-1.55702841  0.56970894  0.31509173 -0.38931675  1.28072173]
exact: 2.2803010541544873e-31, [-1.55702841  0.56970894  0.31509173 -0.38931675  1.28072173]
